In [1]:
import requests
import pandas as pd
import psycopg2

# PostgreSQL connection details
DB_NAME = "crypto_data"
DB_USER = "postgres"
DB_PASSWORD = "1234"
DB_HOST = "localhost"
DB_PORT = "5432"

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
)
cur = conn.cursor()

# API URL and parameters
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 10,
    "page": 1,
    "sparkline": "false"
}

# Fetch data
response = requests.get(url, params=params)
data = response.json()

# Convert to DataFrame and clean
df = pd.DataFrame(data)
df = df[['id', 'symbol', 'name', 'current_price', 'market_cap', 'total_volume']]
df = df.fillna(0)

# Insert data into PostgreSQL
for _, row in df.iterrows():
    cur.execute("""
        INSERT INTO crypto_prices (coin_id, symbol, name, current_price, market_cap, total_volume)
        VALUES (%s, %s, %s, %s, %s, %s)
        """,
        (row['id'], row['symbol'], row['name'], row['current_price'], row['market_cap'], row['total_volume'])
    )

# Commit and close
conn.commit()
cur.close()
conn.close()

print("Data successfully inserted into PostgreSQL!")


Data successfully inserted into PostgreSQL!
